## How To convert a pre-trained PyTorch model to CoreML

In this post we will go through the steps of converting a pre-trained PyTorch model to Apple's CoreML framework. 

Why bother? I don't know about you, but I'd rather not worry about whether my model runs efficiently on platform X, Y, or Z. I just want to focus on solving the ML problem and let someone else figure out how to run it efficiently if the hardware has a GPU or NeuralEngine or whatever they come up with next. This is what CoreML allows us to do on the Apple devices, in theory. Once we have a CoreML model, it will run efficiently on a Mac, iPad, iPhone or Watch. Importing and running the converted CoreML model into an App is supposed to be a breeze. Enough talk, let's find out! 

From [coremltools](https://coremltools.readme.io/docs/pytorch-conversion)
> With coremltools 4.0+, you can convert your model trained in PyTorch to the Core ML format directly, without requiring an explicit step to save the PyTorch model in ONNX format. This is the recommended way to convert your PyTorch model to Core ML format

We can install coremltools via
```terminal
pip install --upgrade coremltools
```


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Define a simple layer module we'll reuse in our network.
class Layer(nn.Module):
    def __init__(self, dims):
        super(Layer, self).__init__()
        self.conv1 = nn.Conv2d(*dims)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, (2, 2))
        return x

In [2]:
# A simple network consisting of several base layers.
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.layer1 = Layer((3, 6, 3))
        self.layer2 = Layer((6, 16, 1))

    def forward(self, input):
        x = self.layer1(input)
        x = self.layer2(x)
        return x

In [3]:
model = SimpleNet()  # Instantiate the network.

In [4]:
example_input = torch.rand(1, 3, 224, 224)  # Example input, needed by jit tracer.
traced_model = torch.jit.trace(model, example_input)  # Generate TorchScript by tracing.

In [5]:
import coremltools as ct
# Convert using the same API. Note that we need to provide "inputs" for pytorch conversion.
model_from_torch = ct.convert(traced_model,
                              inputs=[ct.TensorType(name="input", shape=example_input.shape)])

Translating MIL ==> MLModel Ops: 100%|██████████| 30/30 [00:00<00:00, 23903.71 ops/s]
/Users/ragh/anaconda3/lib/python3.6/site-packages/coremltools/models/model.py:119: RuntimeWarning: You will not be able to run predict() on this Core ML model. Underlying exception message was: Error compiling model: "Error reading protobuf spec. validator error: Layer 'input' produces an output named 'input' which is also an output produced by the layer '__input'.".
  RuntimeWarning,


In [ ]:
# CUSTOMARY IMPORTS
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchaudio
import os
import coremltools as ct

from jumpml import models
from jumpml import eval
from jumpml import SpeechCommandsDataset as scd
from jumpml import utils

import itertools
from IPython.display import Audio

import matplotlib.pyplot as plt
%matplotlib inline

random_seed = 1        
torch.manual_seed(random_seed)
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = "cpu" # quantization is not available on GPU
print('Using', device)

In [6]:
torch.__version__

'1.6.0'

## Step 1: Convert PyTorch model (.pt file) to a TorchScript ScriptModule


### What is TorchScript?
An intermediate representation of a PyTorch model that can be run in C++. We can obtain TorchScript of a PyTorch model (subclass of nn.Module) by  
1. Tracing an existing module
2. Use scripting to directly compile a module  

Tracing is accomplished by creating some sample inputs and then calling the forward method and recording / tracing by a function called torch.jit.trace. The scripting method is useful when there is some control flow (data dependent execution) in the model. We show the tracing method below for our Speech Commands quantized model.

In [ ]:
torch_model = models.SpeechCommandsModel()
example_input = torch.rand(1, 1, 64, 101)
traced_model = torch.jit.trace(torch_model, example_input)
model_from_torch = ct.convert(traced_model,
                              inputs=[ct.TensorType(name="x", shape=example_input.shape)])

In [ ]:
print(torch_model)

In [ ]:
PATH = "./models/speech_commands_model.pt"
nnModel = models.SpeechCommandsModel().to(device)       # Instantiate our model and move model to GPU if available
nnModel.load_state_dict(torch.load(PATH, map_location=torch.device(device)))
nnModel.eval()


In [ ]:
example_input = torch.rand(1, 1, 64, 101)
traced_model = torch.jit.trace(nnModel, example_input)
model_from_torch = ct.convert(traced_model,
                              inputs=[ct.TensorType(name="x", shape=example_input.shape)])

#### MODEL TRACING WITH INPUTS

In [ ]:
testFiles = utils.get_filenames('files',searchstr='SCRIC20*')
(X,y) = scd.get_file_features(testFiles[0], padLR=False)

In [ ]:
# Make a prediction using Core ML
out_dict = model_from_torch.predict({"x": X})

# Print out top-1 prediction
print(out_dict["out"])

#### Convert the model to Core ML using the Unified Conversion API

In [ ]:
import coremltools as ct
# Convert to Core ML using the Unified Conversion API
model = ct.convert(
    traced_model,
    inputs=[ct.TensorType(name="input", shape=X.shape)]
)

In [ ]:
print(traced_model(X))             # TORCHSCRIPT version of QUANTIZED MODEL
print(quantized_model(X))          # QUANTIZED MODEL
print(nnModel(X))                  # ORIGINAL MODEL